In [6]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai 
from pinecone import Pinecone, ServerlessSpec


d9ef6ec8-0d8f-430a-a661-9ac30cc82bc8


In [7]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [8]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Richards',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Dr. Richards is a great teacher who explains complex topics well, but her exams are challenging.'},
 {'professor': 'Dr. Michael Wong',
  'subject': 'Introduction to Psychology',
  'stars': 5,
  'review': 'Dr. Wong is very engaging and makes the subject fascinating. His lectures are a must-attend.'},
 {'professor': 'Dr. Susan Parker',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': "The material is tough, and Dr. Parker's lectures can be a bit dry. Be prepared to study a lot on your own."},
 {'professor': 'Dr. James Thompson',
  'subject': 'World History',
  'stars': 5,
  'review': 'Dr. Thompson is passionate about history and makes every class interesting. Highly recommend!'},
 {'professor': 'Dr. Maria Gonzalez',
  'subject': 'Spanish Literature',
  'stars': 4,
  'review': 'Dr. Gonzalez is very knowledgeable, but the workload is heavy. You will learn a lot, though.'},
 {'professor': '

In [14]:
processed_data = []
openai.api_key = os.getenv("OPENAI_API_KEY")

for review in data['reviews']:
    response = openai.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )

    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [15]:
processed_data[0]

{'values': [-0.0021693683,
  -0.012950966,
  0.025393028,
  0.021634964,
  0.030560365,
  -0.0042310837,
  -0.0030338534,
  0.017054824,
  -0.033117935,
  0.010119369,
  0.02335741,
  0.022822408,
  -0.027350351,
  -0.0047856593,
  -0.0290467,
  0.007274724,
  -0.024153389,
  -0.019312272,
  -0.00015046935,
  0.0648266,
  0.004032089,
  -0.026724009,
  0.06806271,
  0.012285476,
  -0.03327452,
  -0.004766086,
  0.0424348,
  -0.012402914,
  0.03183915,
  0.040607966,
  0.059502676,
  -0.009584367,
  0.028289868,
  -0.0075487494,
  -0.050994836,
  0.03932918,
  -0.015123596,
  0.047784824,
  0.016441528,
  -0.003970107,
  7.243121e-05,
  0.052247524,
  -0.052821673,
  0.009786624,
  0.034709893,
  0.0002501706,
  -0.023383508,
  -0.025667053,
  0.020865083,
  0.028524747,
  -0.07672713,
  0.038050395,
  0.06226903,
  -0.033561595,
  -0.07249931,
  0.037163075,
  0.035179652,
  0.04066016,
  -0.0004991178,
  0.0033600742,
  0.04757604,
  -0.0480458,
  -0.032100126,
  -0.000744599,
  -0.05

In [16]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [17]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}